In [297]:
import numpy as np
import nltk, re
from functions import *


In [284]:
Q = []
Q_all = ''
one,zero = 1,0
i = zero
with open("./cran.qry") as f:
    info = f.read()
    for m in re.finditer(re.compile('.W( )*([a-z -/ \s])*.'), info):
        text = (str)(m.group(0))
        text = text.replace('.W','')
        text = text.replace('.I','')
        i += one
        text = text.replace('\r\n','')
        text = removeStopWords(text,stopWords)
        Q_all = Q_all+' '+text
        Q += [text]
#print(i)

In [285]:
A=[]
A_all=''
j = 0
with open("./cran.all.1400") as f:
    info = f.read()
    for m in re.finditer(re.compile('.W( )*([a-z (.A)(.B) 0-9 \*\+\$\#\^\?\!\:\-\/\.\,\(\)\'\"\= \s])*.I'), info):
        text = (str)(m.group(0))
        text = text.replace('.W','')
        text = text.replace('.I','')
        j += 1
        text = text.replace('\r\n','')
        text = removeStopWords(text,stopWords)
        A.append(text)
        A_all=A_all+text
    to_add = removeStopWords('the buckling shear stress of simply-supported infinitely long plates with transverse stiffeners . this report is an extension of previous theoretical investigations of the elastic buckling in shear of flat plates reinforced by transverse stiffeners . the plates are treated as infinitely long and simply-supported along the long sides . stiffeners are spaced at regular intervals, dividing the plate into a number of panels of uniform size . the effect ob bending and torsional stiffnesses of the stiffener upon the buckling shear stress is calculated for the complete range of stiffnesses, for panels with ratios of width to stiffener spacing of graphical forms .' ,stopWords)
    A += to_add
    A_all = A_all + to_add
#print(j)

In [286]:
Q_token = nltk.word_tokenize(Q_all)
Q_fd = nltk.FreqDist(Q_token)
Q_fd_matrix = np.zeros((len(Q),len(Q_fd)))

In [287]:
Q_wd={}
for word in Q_fd:
    for query in Q:
        if word in query:
            if word in Q_wd:
                Q_wd[word] += one
            else:
                Q_wd[word] = one

In [288]:
Q_idf_seq={}
Q_idf={}
k = zero
for word in Q_wd:
    Q_idf_seq[word ]= k
    Q_idf[word] = np.log(225/Q_wd[word])
    i = 0
    k += one

In [289]:
for query in range(len(Q)):
    temp = nltk.word_tokenize(Q[query])
    #print(temp)
    for token in temp:
        i = 0
        Q_fd_matrix[query][Q_idf_seq[token]] += one

for word in Q_idf:
    Q_fd_matrix[:,Q_idf_seq[word]] = Q_fd_matrix[:,Q_idf_seq[word]] * Q_idf[word]


In [290]:
A_fd = nltk.FreqDist(nltk.word_tokenize(A_all))
A_fd_matrix = np.zeros((len(A),len(Q_fd)))

In [291]:
A_wd = {}

for word in Q_fd:
    counter = zero
    for query in A:
        if word in query:
            if word in A_wd:
                A_wd[word] += one
            else:
                A_wd[word] = one
        else:
            counter += one
    if counter == 1400:
        A_wd[word] = zero

In [292]:
A_idf = {}
A_idf_seq = {}
counter = zero
i = 0
for word in A_wd:
    A_idf_seq[word] = counter
    A_idf[word] = np.log(1400/(A_wd[word]+0.01))
    i += 1
    counter += one

In [293]:
for query in range(len(A)):
    temp = nltk.word_tokenize(A[query])
    for token in temp:
        if token in Q_fd:
            A_fd_matrix[query][A_idf_seq[token]] += one

In [294]:
CSim_matrix = np.zeros((len(Q),len(A)))
for query in range(len(Q)):
    for a in range(len(A)):
        CSim_matrix[query][a] = cosSimilarity(A_fd_matrix[a,:],Q_fd_matrix[query,:])

In [295]:
CSim_matrix = CSim_matrix * 100
A_len=len(abstract)
x = A_len * 0.001
y = x + A_len * 0.002
z = y + A_len * 0.003

In [296]:
output=open('output','w+')
counter = zero
for q in range(len(Q)):
    output.write(str(q+1)+' ')
    i = np.argmax(CSim_matrix[q,:])
    output.write(str(i+1)+' ')
    output.write('-1'+'\n')
    CSim_matrix[q][i] = -1
    for a in range(1,len(A)):
        output.write(str(q+1)+' ')
        index=np.argmax(CSim_matrix[q,:])
        output.write(str(i+1)+' ')
        CSim_matrix[q][i] = -1
        counter += one
        #print(counter)
        if (counter <= x):
            output.write('1'+'\n')
        elif (counter <= y):
            output.write('2'+'\n')
        elif (counter <= z):
            output.write('3'+'\n')
        else:
            output.write('4'+'\n')
output.close() 